## test

In [ ]:
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
#tf.config.run_functions_eagerly(True)
#tf.config.experimental_run_functions_eagerly(True)

## import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Dropout, Conv3D, Flatten
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.models import load_model

## preparation

In [ ]:
bs = 1
epochs = 500
ler = 0.0008

In [ ]:
tr_df = pd.read_csv('I:/poorya/datasets/video dataset/3D/tr_df.csv')
vl_df = pd.read_csv('I:/poorya/datasets/video dataset/3D/vl_df.csv')

In [ ]:
def datagen(folder, df):
    i = 0
    n = df['file name'].tolist()
    l = df['stressor lable'].tolist()

    while (True):
        img = []
        lbl = np.zeros((1, 2))
        lbl[0, 0] = int(l[i][1])
        lbl[0, 1] = int(l[i][3])
        for j in range(40):
            x = image.img_to_array(image.load_img('I:/poorya/datasets/video dataset/3D/' + folder + str(
                n[i]) + '/' + str(j) + '.jpg', color_mode = "grayscale"))
            x /= 255.0
            img.append(x)

        i += 1
        if (i + 1 >= len(n)):
            i = 0
        yield np.array(np.expand_dims(img, axis = 0)), lbl


train_gen = datagen('train/', df = tr_df)
val_gen = datagen('val/', df = vl_df)

## model

In [ ]:
inputt = tf.keras.Input((40, 400, 300, 1))
x = Conv3D(8, (3, 3, 1), activation = 'relu')(inputt)
x = MaxPooling3D(pool_size = (2, 2, 2), strides = (1, 2, 2))(x)
#x = BatchNormalization()(x)
x = Conv3D(16, (3, 3, 1), activation = 'relu')(x)
x = MaxPooling3D(pool_size = (2, 2, 2), strides = (1, 2, 2))(x)
#x = BatchNormalization()(x)
x = Conv3D(32, (3, 3, 1), activation = 'relu')(x)
x = MaxPooling3D(pool_size = (2, 2, 2), strides = (1, 2, 2))(x)
#x = BatchNormalization()(x)
x = Conv3D(64, (3, 3, 1), activation = 'relu')(x)
x = MaxPooling3D(pool_size = (2, 2, 2), strides = (1, 2, 2))(x)
#x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Dropout(0.0)(x)
x = Dense(128)(x)
x = Dropout(0.0)(x)
x = Dense(64)(x)
x = Dropout(0.0)(x)
x = Dense(32)(x)
x = Dropout(0.0)(x)
endd = Dense(2, activation = 'softmax')(x)

model = tf.keras.Model(inputt, endd)

opt = tf.keras.optimizers.Adamax(learning_rate = ler, decay = 0.001, clipvalue = 0.5, epsilon = 1e-07)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(loss = loss, optimizer = opt, metrics = ['accuracy'])
model.summary()

## train

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('video checkpoint', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)
history = model.fit(train_gen,
                    validation_data = val_gen,
                    batch_size = bs,
                    epochs = epochs,
                    verbose = 1,
                    steps_per_epoch = len(os.listdir('I:/poorya/datasets/video dataset/3D/train')) // bs,
                    validation_steps = len(os.listdir('I:/poorya/datasets/video dataset/3D/val')) // bs,
                    callbacks = [checkpoint]
                    )

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

# test

In [ ]:
#test dataset
ts_df = pd.read_csv('I:/poorya/datasets/video dataset/3D/ts_df.csv')
testmodel = load_model('video checkpoint')
tst_gen = datagen('test/', df = ts_df)
tst_loss, tst_acc = testmodel.evaluate(tst_gen)

In [ ]:
#classification report
def change(x):
    answer = np.zeros((np.shape(x)[0]))
    for i in range(np.shape(x)[0]):
        max_value = max(x[i, :])
        max_index = list(x[i, :]).index(max_value)
        answer[i] = max_index
    return answer.astype(np.int)


adrs = 'I:/poorya/datasets/video dataset/3D/test/'
files = ts_df['file name'].tolist()
X_test = [np.load(adrs + files[i]) for i in range(len(files))]
X_test = np.array(X_test)
X_test = [(X_test[i, :] - X_test[i, :].min()) / (X_test[i, :].max() - X_test[i, :].min()) for i in range(len(X_test))]
X_test = np.array(X_test)
Y_test = np.zeros((len(X_test), 2))
Y_tes = ts_df['stressor lable'].tolist()

for i in range(len(X_test)):
    Y_test[i, 0] = int(Y_tes[i][1])
    Y_test[i, 1] = int(Y_tes[i][4])


In [ ]:
Y_pred = testmodel.predict(X_test)
print('\n\n confusion_matrix : \n', confusion_matrix(change(Y_pred), change(Y_test)))
print('\n\n test accuracy is :', accuracy_score(change(Y_pred), change(Y_test)))
print('\n\n classification report : \n', classification_report(change(Y_pred), change(Y_test)))